# Evitement de collision - Création des données

Vous avez maintenant vu comment manœuvrer le robot à l'aide de programmes Python, et à quel point cela est simple ! Mais comment pourrions-nous faire pour que le robot soit capable de se diriger seul ?

C'est un objectif très difficile et de nombreuses approches existent. Mais ce problème peut être décomposé en problématiques plus simples à réaliser. L'une d'entre elles est de faire en sorte que le robot soit capable de détecter un obstacle afin de naviguer en toute sécurité.

Les notebooks contenus dans ce dossier ont pour but de répondre à cette problématique en utilisant comme seul capteur la caméra embarquée du Jetbot ! Vous allez voir comment le Jetbot va pouvoir éviter des obstacles à l'aide d'une IA composée d'un réseau de neurones à convolution.  Bien entendu, la vision du robot est limitée par son champ de vision et donc il ne peut pas détecter des obstacles qu'il ne voit pas.

La procédure que nous allons suivre est très simple :
- Nous allons tout d'abord placer le robot dans des conditions qui ne respectent pas les conditions de "sécurité". Ces conditions seront appelées "bloquer" ("bloquer" est le **label** correspondant à ces conditions). Nous prendrons des photographies à l'aide de la caméra de ce que le robot voit dans ces conditions.

- Ensuite, nous placerons le robot dans des conditions de sécurité. Le label correspondant sera nommé "libre". Comme précédemment, des photographies seront prises et enregistrées sous ce label.

L'ensemble de ces résultats nous permettra de créer un ensemble de données appelé **dataset**. Quand nous aurons beaucoup d'images représentatives de ces deux conditions avec les labels associés, nous enverrons le dataset obtenu dans le GPU du jetson pour **entrainer** le réseau de neurones. Ce réseau de neurones nous permettra ensuite de prédire, en fonction des images issues de la caméra, si le robot se trouve dans une condition de sécurité ou non. Nous utiliserons ensuite le réseau entrainé pour gérer l'évitement de collision pendant la marche du Jetbot.

### Affichage de la caméra

Pour commencer, initialisons la caméra afin de créer un écran de visualisation.

> Le réseau de neurones que nous allons utiliser prend en entrée des images de 224x224 pixels. Nous allons donc configurer la caméra dans cette résolution afin de minimiser la taille des images et donc de notre dataset.
> Dans certaines situations, il sera peut-être préférable de prendre des images plus larges et de les redimensionner par la suite. 

In [ ]:
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224)              # Configure la caméra au format 224x224 pixels

image = widgets.Image(format='jpeg', width=224, height=224)  # Enregistre les images au format 224x224 pixels

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(image)

Maintenant, nous allons créer un répertoire qui nous permettra de sauvegarder nos données. Nous allons nommer ce répertoire ``dataset``. Il va contenir deux sous-répertoires : ``libre`` et ``bloquer``. Les images correspondantes à chaque condition seront sauvegardées dans ces sous-répertoires.

In [ ]:
import os

repertoire_bloquer = 'dataset/bloquer'
repertoire_libre = 'dataset/libre'

# On utilise lex exeptions car si les réperoires exsitent déjà cela provoquera une erreur
try:
    os.makedirs(repertoire_bloquer)
    os.makedirs(repertoire_libre)
except FileExistsError:
    print('Réperoires non créés car ils existent déjà')

Vous devriez maintenant voir le réperoire dataset ainsi que les deux répertoires associés.

Nous allons maintenant afficher quelques boutons afin d'avoir un interface pour prendre les photographies et décider à quelle **classe** (c'est à dire quel label) appartient chaque photo. Nous allons également indiquer combien d'images sont contenues dans chaque classe.

In [ ]:
layout_bouton = widgets.Layout(width='128px', height='64px')
bouton_libre = widgets.Button(description='Ajouter libre', button_style='success', layout=layout_bouton)
bouton_bloquer = widgets.Button(description='Ajouter bloquer', button_style='danger', layout=layout_bouton)
nombre_libre = widgets.IntText(layout=layout_bouton, value=len(os.listdir(repertoire_libre)))
nombre_bloquer = widgets.IntText(layout=layout_bouton, value=len(os.listdir(repertoire_bloquer)))

display(widgets.HBox([nombre_libre, bouton_libre]))
display(widgets.HBox([nombre_bloquer, bouton_bloquer]))

Pour le moment ces boutons n'ont aucune action associée. Il faut que nous attachions des fonctions permettant de sauvegarder une image à chaque fois qu'un clic de souris est détecté sur le bouton. Pour cela nous allons utiliser l'évènement ``on_click``. Nous sauvegarderons alors la valeur du widget ``Image`` (plutôt que celui du widget camera car l'image est déjà compressée au format JPEG !)

Pour être sûr de ne pas répéter le nom des fichiers, nous allons utiliser le package ``uuid``.

In [ ]:
from uuid import uuid1

# Fonction permettant de sauvegarder une image avec un nom de fichier unique
def sauvegarde_image(repertoire):
    chemin_image = os.path.join(repertoire, str(uuid1()) + '.jpg')
    with open(chemin_image, 'wb') as f:
        f.write(image.value)

# Fonction appellée lors de l'appui sur le bouton "Ajouter libre"
def sauvegarde_libre():
    global repertoire_libre, nombre_libre
    sauvegarde_image(repertoire_libre)
    nombre_libre.value = len(os.listdir(repertoire_libre))
    
# Fonction appellée lors de l'appui sur le bouton "Ajouter bloquer"
def sauvegarde_bloquer():
    global repertoire_bloquer, nombre_bloquer
    sauvegarde_image(repertoire_bloquer)
    nombre_bloquer.value = len(os.listdir(repertoire_bloquer))
    
# Création des liens entre les fonctions et les évènements "on_click" des boutons
bouton_libre.on_click(lambda x: sauvegarde_libre())
bouton_bloquer.on_click(lambda x: sauvegarde_bloquer())

Les butons peuvent maintenant sauvegarder nos images dans les bons répertoires ! 

Il est temps de récupérer des données :

1. Placer le robot dans les conditions de non sécurité et cliquer sur le bouton ``Ajouter bloquer``
2. Placer le robot dans les conditions de sécurité et cliquer sur le bouton ``Ajouter libre``
3. Répéter les étapes 1 et 2

Quelques astuces :

1. Utiliser différentes orientations
2. Utiliser différents éclairages
3. Varier les objets à éviter : Murs, objets, meubles, ...
4. Utiliser différentes textures : lisses, granuleuses, verre, ...

Dans l'idéal, plus le dataset est important et plus le robot sera capable d'éviter les collisions dans la vie réelle. Il est important d'avoir des données *variées* et non pas uniquement beaucoup de données. Vous aurez probablement besoin d'au moins 100 images par classe.

In [ ]:
display(image)
display(widgets.HBox([nombre_libre, bouton_libre]))
display(widgets.HBox([nombre_bloquer, bouton_bloquer]))

Fermons maintenant la caméra :

In [ ]:
camera.stop()

## Et après ?

Une fois que les données ont été collectées, nous les utiliserons pour entrainer notre réseau de neurones. Il est possible de compresser les données avec la commande suivante depuis un terminal :

> Le préfixe ! indique que la commande doit être exécutée dans un terminal (ou *shell*)

> L'argument -r dans la commande zip précise qu'on souhaite compresser les fichiers de manière récursive (tous les fichiers dans les sous-répertoires seront donc sont pris en compte).

In [ ]:
!zip -r dataset.zip dataset

Vous devriez voir un fichier nommé ``dataset.zip`` dans l'explorateur de fichiers.  Vous pouvez télécharger ce fichier en cliquant sur ``Download``.